## Getting consensus sequences from alleles.loci files

**20170717**

I want to build a FASTA file that has the most common allele per locus across my RAD data, to use as a "reference genome" in ipyrad. I'm going to put together that code in this notebook, and ultimately put it into a script that I can reuse.

In [1]:
cd /mnt/hgfs/SHARED_FOLDER

/mnt/hgfs/SHARED_FOLDER


In [2]:
# get blocks of alleles separated by locus
test = open("test_all_loci.txt","r")
blocks = []
b = []
for line in test:
    if line.startswith('//'):
        if len(b) > 0:
            b.append(line)
            blocks.append(b)
        b = []
    else:
          b.append(line)
test.close()

In [3]:
# get index of starting base
firstline = blocks[0][0].strip()

# define function gets indeces of every time character appears in string
def find(s, ch):
    return [i for i, ltr in enumerate(s) if ltr == ch]

# get indeces of spaces in firstline
base_start = find(firstline, " ")[-1]+1

In [6]:
base_start

14

In [4]:
ex_hash = blocks[0][-1].strip()

In [5]:
ex_hash

'//                                        -            -                 *    *                     *                         * -                         |5|'

In [8]:
# get indeces with SNPs
seq_mark = ex_hash.split("|")[0][14:]
seq_mark

'                            -            -                 *    *                     *                         * -                         '

In [9]:
char_ind = 0
snp_inds = []
for char in seq_mark:
    if char != " ":
        snp_inds.append(char_ind)
    char_ind += 1

In [11]:
snp_inds

[28, 41, 59, 64, 86, 112, 114]

In [60]:
import pandas as pd
import numpy as np

In [19]:
# try making data frame out of alignment within locus block with pandas
aln_list_for_df = []
for line in blocks[0][:-1]:
    aln_list_for_df.append(list(line.strip().split()[1]))
block_df = pd.DataFrame(aln_list_for_df)

In [24]:
block_df.iloc[0:5,50:65] # check it out and make sure dashes show up

,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64
0,A,C,C,A,G,G,A,C,C,T,T,A,T,A,T
1,A,C,C,A,G,G,A,C,C,T,T,A,T,A,T
2,A,C,C,A,G,G,A,C,C,T,-,-,T,A,T
3,A,C,C,A,G,G,A,C,C,T,-,-,T,A,T
4,A,C,C,A,G,G,A,C,C,T,-,-,T,A,T


In [71]:
length = len(list(block_df.mode()))
sum_seq = ""
for i in np.arange(0,length):
    sum_seq += block_df.mode()[i][0]


In [72]:
print sum_seq

TAAAGCTGTATAGTTGGAGGTGTCTGGGCCTTGTCCCAGCTAGCTAGCTAACCAGGACCT--TATATGTATGGTGTGATGTCCAGATCAGGTCAAGCCTATTTCTGTCCCTACGAAGGACAGAAGCAGTGACCAGTATCT


In [83]:
sum_seq.replace("-","")

'TAAAGCTGTATAGTTGGAGGTGTCTGGGCCTTGTCCCAGCTAGCTAGCTAACCAGGACCTTATATGTATGGTGTGATGTCCAGATCAGGTCAAGCCTATTTCTGTCCCTACGAAGGACAGAAGCAGTGACCAGTATCT'

In [76]:
y = [['A','T','C','G'],['A','C','C','G'],['C','G','C','G'],['C','G','C','G']]
y_df = pd.DataFrame(y)
y_df

,0,1,2,3
0,A,T,C,G
1,A,C,C,G
2,C,G,C,G
3,C,G,C,G


In [80]:
y_df.mode()

,0,1,2,3
0,A,G,C,G
1,C,NaN,NaN,NaN


In [82]:
y_df.mode()[1][1]

nan

In [89]:
cd /mnt/hgfs/SHARED_FOLDER/Learn_iPyrad/CRAGIG_RUNS_12_py/CG_l1_25_c82_H_outfiles

/mnt/hgfs/SHARED_FOLDER/Learn_iPyrad/CRAGIG_RUNS_12_py/CG_l1_25_c82_H_outfiles


In [ ]:
# get blocks of alleles separated by locus
infile = open("CG_l1_25_c82_H.alleles.loci","r")
blocks = []
b = []
for line in infile:
    if line.startswith('//'):
        if len(b) > 0:
            b.append(line)
            blocks.append(b)
        b = []
    else:
          b.append(line)
infile.close()

outfile = open("refgen_cragig.fasta","w")

for block in blocks:
    for_df = []
    for line in block[:-1]:
        for_df.append(list(line.strip().split()[1]))
    block_df = pd.DataFrame(for_df)
    length = len(list(block_df.mode()))
    sum_seq = ""
    for i in np.arange(0,length):
        sum_seq += block_df.mode()[i][0]
    locus_num = block[-1].split("|")[1]
    clean_seq = sum_seq.replace("-","")
    outfile.write(">" + locus_num + "\n" + clean_seq + "\n")

outfile.close()